ZAČETNE NASTAVITVE

In [73]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

pd.options.display.max_rows = 100

cv=pd.read_csv("Podatki.csv", encoding_errors="replace")

#spremeni slabe podatke
cv["Stevilo_zgrajenih"] = cv["Stevilo_zgrajenih"].str.replace(" ", "").astype(float)
cv["Leto_prvega_leta"]=cv["Leto_prvega_leta"].astype("str").str.replace("?", "NaN").astype(float)

ANALIZA IN PREDSTAVITEV PODATKOV

In [74]:
#naredi nov DataFrame in razvrsti podatke glede na št izdelanih za 10 najbolj izdelanih
nmade_frame=pd.DataFrame({"Ime_letala" : cv.sort_values(by=["Stevilo_zgrajenih"], ascending=False)["Ime_letala"].head(10),
                          "Stevilo_zgrajenih" : cv.sort_values(by=["Stevilo_zgrajenih"], ascending=False)["Stevilo_zgrajenih"].head(10)})
nmade_frame.loc[len(nmade_frame.index)] = ["Ostali", cv["Stevilo_zgrajenih"].sum()-nmade_frame["Stevilo_zgrajenih"].sum()]
nmade_frame=nmade_frame.sort_values(by="Stevilo_zgrajenih")

#nariši graf
g_bar=px.bar(nmade_frame, x="Stevilo_zgrajenih", y="Ime_letala", title="Najbolj izdelana letala".upper(),
       labels=dict(Stevilo_zgrajenih="Število zgrajenih", Ime_letala="Ime letala"), range_x=[0, 50000])
g_bar.update_layout(xaxis=dict(tickvals=[0, 10000, 20000, 30000, 40000, 50000], ticktext=["0", "10k", "20k", "30k", "40k", ">50k"]))
g_bar.show()

In [75]:
#ustvari Dataframe in vpiši št zgrajenih letal glede na države
nmap_frame=pd.DataFrame(cv.groupby("Drzava_porekla")['Stevilo_zgrajenih'].sum().sort_values(ascending=False).reset_index())
nmap_frame.rename(columns={"Drzava_porekla" : "Država Porekla", "Stevilo_zgrajenih" : "Število izdelanih"}, inplace=True)

#graf
nmap = px.choropleth(nmap_frame, locations="Država Porekla", locationmode="country names", color="Število izdelanih",  
                     color_continuous_scale="ylgnbu", range_color=[0, 50000], scope="world", title="Število izdelanih letal glede na države".upper()) 
nmap.show()


In [76]:
#DF za top20 izdelovalcev in št narejenih letal
manu_frame=pd.DataFrame(cv.groupby('Izdelovalec')['Stevilo_zgrajenih'].sum().sort_values(ascending=False)).reset_index().head(20)

#graf
table=go.Figure(data=[go.Table(header=dict(values=["Izdelovalec", "Število izdelanih letal"], line_color="black", fill_color="#FF6692"),
                               cells=dict(values=[manu_frame["Izdelovalec"], manu_frame["Stevilo_zgrajenih"]], line_color="black", fill_color="#fccde5"))])
table.update_layout(title_text="DVAJSET NAJVEČJIH PROIZVAJALCEV GLEDE NA ŠTEVILO IZDELANIH LETAL")

table.show()

In [77]:
#DataFrame za podatke o številu modelov glede na države
wmap_frame=pd.DataFrame(cv[["Drzava_porekla"]].value_counts().reset_index())
wmap_frame.rename(columns={"Drzava_porekla" : "Država Porekla", "count" : "Število modelov"}, inplace=True)

#graf
wmap = px.choropleth(wmap_frame, locations="Država Porekla", locationmode="country names", color="Število modelov", 
                     title="Število modelov letal glede na države".upper(), color_continuous_scale="Purpor", range_color=[0, 500], scope="world")
wmap.update_layout(coloraxis_colorbar=dict(tickvals=[0, 100, 200, 300, 400, 500], ticktext=["0", "100", "200", "300", "400", ">500"])) 
wmap.show()


In [78]:
#naredi DF za desetletja in samo top 5 držav + ostali
r_frame=pd.DataFrame({"Leto_prvega_leta" : (cv["Leto_prvega_leta"] // 10 * 10).astype("Int64"), "Drzava_porekla" : cv["Drzava_porekla"]}) #DF za desetletja in države

new_frame=pd.DataFrame(r_frame.groupby("Leto_prvega_leta").value_counts()).reset_index() #DF za desetletje in skupno modelov po državah
new_frame=new_frame.pivot(index="Drzava_porekla", columns="Leto_prvega_leta", values="count").drop("_unknown_", axis=0) #zamenjaj stolpce... in odstrani unknown

r_frame=r_frame.groupby('Leto_prvega_leta')["Drzava_porekla"].size().sort_index(ascending=True).astype(float).reset_index() #skupno modelov po desetletjih
for co in new_frame.columns: #ohrani samo vrednostni za top 5 držav po desetletjih
    new_frame[co] = new_frame.where(new_frame.eq(new_frame.sort_values(by=co, ascending=False).head(5)))[co]

new_frame=new_frame.transpose().dropna(axis=1, how="all") 
new_frame["Druge"] = r_frame["Drzava_porekla"].values - new_frame.sum(axis=1, numeric_only=True).values #dodaj ostalo 
new_frame.index.name="Leto prvega leta"
new_frame.columns.name="Država porekla"

#graf
px.bar(new_frame, x=new_frame.index, y=new_frame.columns, color_discrete_sequence=px.colors.qualitative.Prism, 
       title="Število modelov letal po državah glede na desetletje".upper(), labels={"value" : "Število modelov"})


In [79]:
#DF za vlogo po časovni dobi
role_frame=pd.DataFrame({"1900" : cv.query("Leto_prvega_leta >= 1900  and Leto_prvega_leta <= 1990").value_counts("Vloga") , "1991" : cv.query("Leto_prvega_leta >= 1991 ").value_counts("Vloga")})

#procent spremembe od 1900 do 1991
role_frame["p1900"] = role_frame["1900"] / role_frame["1900"].sum() * 100
role_frame["p1991"] = role_frame["1991"] / role_frame["1991"].sum() * 100
role_frame["Sprememba"] = ((role_frame["p1991"] - role_frame["p1900"]) / role_frame["p1900"]).round(2).fillna(-1)
role_frame["Colour"] = np.where(role_frame["Sprememba"] >= 0, "#636EFA", "#EF553B") #barve grafa

#grafi
pie=make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], subplot_titles=["1900 - 1990", "1991-2019"])
pie.add_trace(go.Pie(labels=role_frame.index, values=role_frame["1900"], name=""), 1, 1)
pie.add_trace(go.Pie(labels=role_frame.index, values=role_frame["1991"], name=""), 1, 2)
pie.update_layout(title_text="Procent novih modelov po vlogah med časovnimi obdobji".upper())
pie.show()

g_bar2=px.bar(role_frame["Sprememba"], title="SPREMEMBA VLOG NOVIH MODELOV LETAL IZ OBDOBJA (1900-1990) K (1991-2019)", 
              labels={"value" : "Sprememba"})
g_bar2.update_traces(marker_color=role_frame["Colour"], hovertemplate=None)
g_bar2.update_layout(showlegend=False, hovermode="x")
g_bar2.show()
